In [1]:
import random
import math
import numpy as np
import matplotlib.pyplot as plt
# import scipy.io

Instructions: Each element in the array contains data on one trial. Time stamps for each spike in trial i and neuron j are given in r(i).unit(j).spikeTimes (in ms relative to trial start). Other relevant members of the structure for this homework assignment are: r(i).timeTouchHeld (the time the reach target appears and movement planning nominally begins), r(i).timeGoCue (the time the animal is cued to move and planning nominally ends), r(i).timeTargetAcquire (the time movement ends).

In [2]:
spikeTimes = np.load('spikeTimes.npy', allow_pickle=True)
timeGoCue = np.load('timeGoCue.npy', allow_pickle=True)
timeTargetAcquire = np.load('timeTargetAcquire.npy', allow_pickle=True)
timeTouchHeld = np.load('timeTouchHeld.npy', allow_pickle=True)
cfr = np.load('cfr.npy', allow_pickle=True)

In [3]:
print(timeTouchHeld[0])
print(timeGoCue[0])
print(timeTargetAcquire[0])
print(spikeTimes[0][0]) # Claro, porque la neurona 0 disparo 3 veces, en la prueba 0.
# Y los tiempos que dice ahi es cuando ocurrio eso, que fue entre timeTouchHeld y timeGoCue
# print(spikeTimes)

480
1485
2005
[549.51354167 595.64166667 669.74583333]


In [4]:
# Example code for finding the number of spikes in the plan window

planSpikes = []

for trialIdx, trialSpikes in enumerate(spikeTimes): #what does st mean?
    planSpikes.append([np.sum((st > timeTouchHeld[trialIdx]) & (st < timeGoCue[trialIdx])) for st in trialSpikes])

planSpikes = np.array(planSpikes) # will be 1127 x 190 (number of trials by number of neurons)
planSpikes

array([[ 3,  8,  1, ...,  7,  8, 13],
       [ 1,  6,  0, ...,  4,  9, 12],
       [ 1,  3,  2, ...,  9, 11, 17],
       ...,
       [ 0,  7,  0, ...,  7,  3,  5],
       [ 0,  4,  0, ...,  8,  4, 11],
       [ 2,  9,  0, ..., 12,  3,  8]])

In [5]:
# Modified example code for finding the number of spikes in the movement window

movementSpikes = []

for trialIdx, trialSpikes in enumerate(spikeTimes): #what does st mean?
     movementSpikes.append([np.sum((st > timeGoCue[trialIdx]) & (st < timeTargetAcquire[trialIdx])) for st in trialSpikes])

movementSpikes = np.array(movementSpikes) # will be 1127 x 190 (number of trials by number of neurons)
movementSpikes

array([[ 0,  8,  3, ...,  7,  5, 19],
       [ 0,  5,  1, ...,  1, 20, 14],
       [ 0,  8,  1, ...,  2, 10, 21],
       ...,
       [ 0,  5,  0, ...,  3, 13, 19],
       [ 0,  7,  0, ...,  1, 13,  3],
       [ 0,  5,  2, ...,  3,  7, 10]])

### Obtaining the rates (lambdas) for each neuron in each trial in the plan, movement and trial-movement window

In [6]:
trials = 1127   # 1127 trials
neurons = 190  # 190 neurons

lambda_planSpikes = np.zeros((1127,190))
lambda_movementSpikes = np.zeros((1127,190))
lambda_plan_move_Spikes = np.zeros((1127,190))

for trial_i in range(trials):
    for neuron_i in range(neurons):
        lambda_planSpikes[trial_i][neuron_i] = planSpikes[trial_i][neuron_i]/(timeGoCue[trial_i]-timeTouchHeld[trial_i])
        lambda_movementSpikes[trial_i][neuron_i] = movementSpikes[trial_i][neuron_i]/(timeTargetAcquire[trialIdx]-timeGoCue[trial_i])
        
        #Combined data
        spikes = planSpikes[trial_i][neuron_i] + movementSpikes[trial_i][neuron_i]
        time_window = (timeGoCue[trial_i]-timeTouchHeld[trial_i]) + (timeTargetAcquire[trialIdx]-timeGoCue[trial_i])
        lambda_plan_move_Spikes[trial_i][neuron_i] = spikes/time_window
        
lambda_planSpikes        
# lambda_movementSpikes
# lambda_plan_move_Spikes

array([[0.00298507, 0.0079602 , 0.00099502, ..., 0.00696517, 0.0079602 ,
        0.01293532],
       [0.00099502, 0.00597015, 0.        , ..., 0.0039801 , 0.00895522,
        0.0119403 ],
       [0.00099502, 0.00298507, 0.00199005, ..., 0.00895522, 0.01094527,
        0.01691542],
       ...,
       [0.        , 0.00927152, 0.        , ..., 0.00927152, 0.00397351,
        0.00662252],
       [0.        , 0.0039801 , 0.        , ..., 0.0079602 , 0.0039801 ,
        0.01094527],
       [0.00199005, 0.00895522, 0.        , ..., 0.0119403 , 0.00298507,
        0.0079602 ]])

In [7]:
# Training data
# The cfr has the objective of each trial.
# Randomly choose 50 data sets for each direction, that is 400 sets in total.

# Indices of each objective
vec_1 = [i for i in range(len(cfr)) if cfr[i] == 1]
vec_2 = [i for i in range(len(cfr)) if cfr[i] == 2]
vec_3 = [i for i in range(len(cfr)) if cfr[i] == 3]
vec_4 = [i for i in range(len(cfr)) if cfr[i] == 4]
vec_5 = [i for i in range(len(cfr)) if cfr[i] == 5]
vec_6 = [i for i in range(len(cfr)) if cfr[i] == 6]
vec_7 = [i for i in range(len(cfr)) if cfr[i] == 7]
vec_8 = [i for i in range(len(cfr)) if cfr[i] == 8]

# Training indices for each objective
training_1 = random.sample(vec_1, 50)
training_2 = random.sample(vec_2, 50)
training_3 = random.sample(vec_3, 50)
training_4 = random.sample(vec_4, 50)
training_5 = random.sample(vec_5, 50)
training_6 = random.sample(vec_6, 50)
training_7 = random.sample(vec_7, 50)
training_8 = random.sample(vec_8, 50)

In [8]:
# Organizing the training and testing indices
training_indices = np.column_stack((training_1, training_2, training_3, training_4,
                                    training_5, training_6, training_7, training_8))
training_indices = training_indices.reshape(400)
training_indices = np.sort(training_indices)
training_indices

testing_indices = []

for i in range(trials):
    if i not in training_indices:
        testing_indices.append(i)

print(np.size(training_indices))
print(np.size(testing_indices))

400
727


In [9]:
# Obtaining the lambdas we will use for evaluating in the Poisson model
# There are lambdas for planning, movement and plan_movement epochs.

lambda_plan = np.zeros((8, 190))
lambda_movement = np.zeros((8, 190))
lambda_plan_move = np.zeros((8, 190))

for neuron_i in range(190):
    for i in training_1: #Cycle for training data in 1st direction
        lambda_plan[0][neuron_i] += lambda_planSpikes[i][neuron_i] # Lambda for neuron i
        lambda_movement[0][neuron_i] += lambda_movementSpikes[i][neuron_i] # Lambda for neuron i
        lambda_plan_move[0][neuron_i] += lambda_plan_move_Spikes[i][neuron_i] # Lambda for neuron i
    
    for i in training_2: #Cycle for training data in 2nd direction
        lambda_plan[1][neuron_i] += lambda_planSpikes[i][neuron_i] # Lambda for neuron i
        lambda_movement[1][neuron_i] += lambda_movementSpikes[i][neuron_i] # Lambda for neuron i
        lambda_plan_move[1][neuron_i] += lambda_plan_move_Spikes[i][neuron_i] # Lambda for neuron i

    for i in training_3: #Cycle for training data in 3rd direction
        lambda_plan[2][neuron_i] += lambda_planSpikes[i][neuron_i] # Lambda for neuron i
        lambda_movement[2][neuron_i] += lambda_movementSpikes[i][neuron_i] # Lambda for neuron i
        lambda_plan_move[2][neuron_i] += lambda_plan_move_Spikes[i][neuron_i] # Lambda for neuron i

    for i in training_4: #Cycle for training data in 4th direction
        lambda_plan[3][neuron_i] += lambda_planSpikes[i][neuron_i] # Lambda for neuron i
        lambda_movement[3][neuron_i] += lambda_movementSpikes[i][neuron_i] # Lambda for neuron i
        lambda_plan_move[3][neuron_i] += lambda_plan_move_Spikes[i][neuron_i] # Lambda for neuron i

    for i in training_5: #Cycle for training data in 5th direction
        lambda_plan[4][neuron_i] += lambda_planSpikes[i][neuron_i] # Lambda for neuron i
        lambda_movement[4][neuron_i] += lambda_movementSpikes[i][neuron_i] # Lambda for neuron i
        lambda_plan_move[4][neuron_i] += lambda_plan_move_Spikes[i][neuron_i] # Lambda for neuron i

    for i in training_6: #Cycle for training data in 6th direction
        lambda_plan[5][neuron_i] += lambda_planSpikes[i][neuron_i] # Lambda for neuron i
        lambda_movement[5][neuron_i] += lambda_movementSpikes[i][neuron_i] # Lambda for neuron i
        lambda_plan_move[5][neuron_i] += lambda_plan_move_Spikes[i][neuron_i] # Lambda for neuron i

    for i in training_7: #Cycle for training data in 7th direction
        lambda_plan[6][neuron_i] += lambda_planSpikes[i][neuron_i] # Lambda for neuron i
        lambda_movement[6][neuron_i] += lambda_movementSpikes[i][neuron_i] # Lambda for neuron i
        lambda_plan_move[6][neuron_i] += lambda_plan_move_Spikes[i][neuron_i] # Lambda for neuron i

    for i in training_8: #Cycle for training data in 8th direction
        lambda_plan[7][neuron_i] += lambda_planSpikes[i][neuron_i] # Lambda for neuron i
        lambda_movement[7][neuron_i] += lambda_movementSpikes[i][neuron_i] # Lambda for neuron i
        lambda_plan_move[7][neuron_i] += lambda_plan_move_Spikes[i][neuron_i] # Lambda for neuron i

    # You could continue making one lambda array for each direction for neuron i
    
    for i in range(8):
        lambda_plan[i] = lambda_plan[i]/50
        lambda_movement[i] = lambda_movement[i]/50
        lambda_plan_move[i] = lambda_plan_move[i]/50
        
print(lambda_plan) #8*190 = 1520
print(lambda_movement) #8*190
print(lambda_plan_move) # 8*190

[[0.00000000e+000 1.77863633e-322 4.70844560e-321 ... 3.49157524e-006
  1.23709927e-004 7.88771375e-003]
 [0.00000000e+000 1.97626258e-322 5.34579029e-321 ... 2.97737801e-006
  1.48602682e-004 9.37972390e-003]
 [0.00000000e+000 1.43279037e-322 5.38531554e-321 ... 3.76517413e-006
  2.20753188e-004 7.38413891e-003]
 ...
 [0.00000000e+000 2.27270197e-322 2.14424490e-321 ... 3.01174920e-006
  2.31085631e-004 7.56970116e-003]
 [0.00000000e+000 2.27270197e-322 2.90016534e-321 ... 3.34734276e-006
  1.19171032e-004 9.38433660e-003]
 [0.00000000e+000 2.27270197e-322 2.36657444e-321 ... 3.23136635e-006
  1.40626668e-004 7.86030114e-003]]
[[0.00000000e+000 6.27463370e-322 6.40803143e-321 ... 2.02312450e-006
  5.13829186e-004 1.93740148e-002]
 [0.00000000e+000 6.42285340e-322 5.62740771e-321 ... 2.36637410e-006
  4.13388172e-004 2.81086046e-002]
 [0.00000000e+000 7.06513874e-322 1.02172776e-320 ... 3.62924348e-006
  8.97192119e-004 3.06128316e-002]
 ...
 [0.00000000e+000 6.47225996e-322 3.05332569

At least until here, it is running okey.

In [10]:
# Likelihood equation
def Poisson(x_d, lambda_d): #This formula is only for the testing data

    if lambda_d < 1e-100: # Very small lambdas return a value that goes to infty.
        log_term = -230*x_d
    else:
        log_term = x_d*np.log(lambda_d)
        
#     if x_d >20: # For more than 20 spikes, the log(spikes!) comes with an error message
#         # AttributeError: 'int' object has no attribute 'log'
#         # np.log(math.factorial(20)) # For numbers greater than 20, there comes this error message
#         factorial = np.log(np.double(np.math.factorial(x_d)))
#     else:
#         factorial = np.log(math.factorial(x_d))
        
    return -lambda_d + log_term # - factorial
# Note the factorial term does not matter because it is the same term for each direction, so when you compare them,
# it does not make a difference

In [11]:
# This sample code is for running ONE trial.
# testing_i = testing_indices[2]
# aux_plan = np.zeros(8)
# aux_move = np.zeros(8)

# for direction_i in range(8):
#     for neuron_i in range(neurons):
#         aux_plan[direction_i] += Poisson(planSpikes[testing_i][neuron_i], lambda_d[direction_i][neuron_i])
#         aux_move[direction_i] += Poisson(movementSpikes[testing_i][neuron_i], lambda_movementSpikes[direction_i][neuron_i])
        
        
# print(aux_plan)
# print(np.argmax(aux_plan)+1)

# print("For movement")
# print(aux_move)
# print(np.argmax(aux_move)+1)

In [12]:
# Poisson values for the planning and movement data
direction_values_plan = np.zeros(len(testing_indices))
direction_values_move = np.zeros(len(testing_indices))
direction_values_plan_move = np.zeros(len(testing_indices))
i = 0

for testing_i in testing_indices:
    aux_plan = np.zeros(8)
    aux_move = np.zeros(8)
    aux_plan_move = np.zeros(8)
    
    for direction_i in range(8):
        for neuron_i in range(neurons):

            aux_plan[direction_i] += Poisson(lambda_planSpikes[testing_i][neuron_i], lambda_plan[direction_i][neuron_i])
            aux_move[direction_i] += Poisson(lambda_movementSpikes[testing_i][neuron_i], lambda_movement[direction_i][neuron_i])
            aux_plan_move[direction_i] += Poisson(lambda_plan_move_Spikes[testing_i][neuron_i], lambda_plan_move[direction_i][neuron_i])
              
    direction_values_plan[i] = np.argmax(aux_plan)+1
    direction_values_move[i] = np.argmax(aux_move)+1
    direction_values_plan_move[i] = np.argmax(aux_plan_move)+1
    
    i+=1 # This is only an index for the testing_indices
    
print(direction_values_plan)
print(direction_values_move)
print(direction_values_plan_move)
# Notice you did a change here. The last 3 arrays you called cannot be called with the same index as the original values.

[6. 3. 6. 6. 6. 1. 3. 3. 1. 3. 6. 6. 7. 6. 3. 3. 3. 3. 7. 3. 1. 3. 6. 3.
 3. 3. 3. 1. 1. 6. 1. 3. 3. 6. 6. 6. 3. 3. 1. 3. 6. 3. 3. 3. 1. 3. 1. 6.
 6. 3. 6. 3. 3. 3. 6. 3. 1. 6. 3. 1. 3. 6. 3. 3. 3. 6. 3. 6. 3. 1. 3. 3.
 3. 6. 6. 6. 3. 6. 3. 3. 6. 3. 1. 6. 6. 3. 3. 3. 3. 3. 3. 3. 3. 6. 3. 3.
 6. 6. 3. 6. 3. 3. 3. 6. 3. 3. 3. 1. 3. 3. 3. 3. 6. 3. 6. 3. 3. 3. 6. 3.
 3. 1. 6. 3. 3. 3. 3. 3. 6. 3. 3. 6. 3. 1. 3. 3. 6. 3. 6. 3. 6. 7. 6. 3.
 1. 3. 3. 3. 3. 3. 3. 1. 6. 6. 3. 1. 3. 6. 3. 3. 1. 6. 3. 6. 3. 3. 3. 3.
 3. 6. 6. 3. 1. 3. 1. 6. 3. 3. 3. 3. 6. 6. 3. 3. 1. 3. 3. 6. 3. 3. 3. 3.
 1. 6. 7. 3. 6. 3. 3. 3. 3. 3. 3. 1. 1. 6. 1. 3. 6. 3. 3. 1. 1. 3. 3. 6.
 6. 6. 3. 3. 3. 3. 3. 3. 1. 3. 1. 3. 6. 3. 6. 6. 3. 3. 3. 3. 6. 6. 3. 6.
 3. 7. 3. 3. 3. 3. 6. 3. 6. 3. 6. 3. 3. 3. 3. 6. 3. 3. 3. 3. 3. 3. 3. 3.
 6. 6. 3. 3. 6. 1. 1. 3. 1. 3. 6. 6. 3. 1. 3. 6. 3. 3. 3. 3. 3. 1. 6. 1.
 3. 1. 1. 6. 6. 6. 3. 1. 3. 3. 6. 3. 6. 6. 3. 3. 3. 3. 3. 3. 3. 6. 6. 6.
 3. 3. 3. 1. 3. 6. 3. 3. 3. 3. 1. 1. 3. 6. 3. 6. 3.

In [13]:
# Correctness of trials
i = 0
correct_trials_plan = 0
correct_trials_move = 0
correct_trials_plan_move = 0

# Yes, you must call them with different indices because they have different sizes.
# But you did the correct mapping.
for testing_i in testing_indices:
    
    if cfr[testing_i] == direction_values_plan[i]:
        correct_trials_plan += 1
    
    if cfr[testing_i] == direction_values_move[i]:
        correct_trials_move += 1
    
    if cfr[testing_i] == direction_values_plan_move[i]:
        correct_trials_plan_move += 1
    
    i += 1

print("Correct trials in planning: ", correct_trials_plan/len(testing_indices))
print("Correct trials in movement: ", correct_trials_move/len(testing_indices))
print("Correct trials in planning and movement: ", correct_trials_plan_move/len(testing_indices))
print("Chance: ",1/8)

Correct trials in planning:  0.2764786795048143
Correct trials in movement:  0.1279229711141678
Correct trials in planning and movement:  0.3314993122420908
Chance:  0.125


# BEWARE if you are getting best decoding numbers below 90% you probably have not properly calculated rates or properly applied the rates to the right duration windows for each trial. Remember that a Poisson process is defined by λ∗T!

In [14]:
# Use the training data to estimate the parameters of a target- direction-dependent vector Poisson process
# calculate a Poisson model for each neuron and class - the rate for a class will be a vector the size of the number of neurons.

# How do we estimate a Poisson model with the given data?

